### Optimization Project 1 :Code

#### Chih-En Ko (ck29388)
#### Eeshana Hamed (eh28497)
#### Mandeep Burdak (msb4384)
#### Sanyam Jain (sj33448)

In [1]:
import gurobipy as gp
import pandas as pd

# Read ROI Data
roi_data = pd.read_csv('ROI_data.csv')
roi_data.drop(roi_data.columns[0], axis=1, inplace=True)
ncol = len(roi_data.columns)

In [2]:
roi_data

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,0.031,0.049,0.024,0.039,0.016,0.024,0.046,0.026,0.033,0.044
1,0.049,0.023,0.024,0.039,0.044,0.046,0.026,0.019,0.037,0.026


In [3]:
# Read Month Data
roi_data_month = pd.read_csv('roi_mat.csv')
months = roi_data_month.T.iloc[0].tolist()
roi_data_month.drop(roi_data_month.columns[0], axis=1, inplace=True)
ncol_month = len(roi_data_month.columns)
nrow_month = len(roi_data_month)

In [4]:
roi_data_month

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,4.0,3.6,2.4,3.9,3.0,3.5,3.6,2.25,3.5,3.5
1,4.0,3.9,2.7,3.8,4.3,3.2,2.7,1.80,3.7,3.5
2,3.5,2.9,3.1,3.8,2.4,4.1,3.7,2.60,4.2,2.5
3,3.8,3.1,2.4,4.4,2.4,3.8,3.7,2.50,3.6,2.9
4,3.5,3.2,1.9,3.4,2.7,2.7,3.9,2.20,4.5,3.9
5,4.0,3.2,2.7,3.4,3.4,3.0,4.5,2.10,3.8,4.1
6,3.9,3.6,2.0,4.4,3.9,3.7,4.3,1.80,4.0,3.8
7,4.2,3.3,2.8,4.2,2.0,3.7,3.6,1.50,4.4,4.3
8,4.1,2.8,2.5,4.2,2.9,3.7,2.8,2.50,4.0,3.4
9,3.0,3.0,3.1,4.6,3.1,3.3,3.2,2.30,2.5,3.2


## Specific 1,2,3

In [5]:
platform = roi_data.columns.tolist()
roi1Mod = gp.Model()

# Variables (also setting the upper bound of each variable to $3M i.e. constraint 3)
x = roi1Mod.addVars(platform, ub=3e6, name = platform)

# Objective function
roi1Mod.setObjective(gp.quicksum(roi_data[plat][0] * x[plat] for plat in platform), gp.GRB.MAXIMIZE)

# Constraints
roi1Mod.addConstr(x.sum() <= 10e6) # Total budget constraint
roi1Mod.addConstr(x['Print'] + x['TV'] <= x['Facebook'] + x['Email']) # Constraint 1
roi1Mod.addConstr(x['Facebook'] + x['LinkedIn'] + x['Instagram'] + x['Snapchat'] + x['Twitter'] >= 2*(x['SEO'] + x['AdWords'])) # Constraint 2

# Optimize
roi1Mod.Params.OutputFlag = 0
roi1Mod.optimize()

# Result
for i in range(ncol):
    print(f'{roi1Mod.varName[i]}: ${roi1Mod.X[i]:.2f}')

print(f'Optimal ROI: ${roi1Mod.objVal:.2f}')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-20
Print: $0.00
TV: $3000000.00
SEO: $0.00
AdWords: $1000000.00
Facebook: $0.00
LinkedIn: $0.00
Instagram: $3000000.00
Snapchat: $0.00
Twitter: $0.00
Email: $3000000.00
Optimal ROI: $456000.00


## Specific 4

In [6]:
roi2Mod = gp.Model()

# Variables (also setting the upper bound of each variable to $3M i.e. constraint 3)
x = roi2Mod.addVars(platform, ub=3e6, name = platform)

# Objective function
roi2Mod.setObjective(gp.quicksum(roi_data[plat][1] * x[plat] for plat in platform), gp.GRB.MAXIMIZE)

# Constraints
roi2Mod.addConstr(x.sum() <= 10e6) # Total budget constraint
roi2Mod.addConstr(x['Print'] + x['TV'] <= x['Facebook'] + x['Email']) # Constraint 1
roi2Mod.addConstr(x['Facebook'] + x['LinkedIn'] + x['Instagram'] + x['Snapchat'] + x['Twitter'] >= 2*(x['SEO'] + x['AdWords'])) # Constraint 2


# Optimize
roi2Mod.Params.OutputFlag = 0
roi2Mod.optimize()

# Result
for i in range(ncol):
    print(f'{roi2Mod.varName[i]}: ${roi2Mod.X[i]:.2f}')

print(f'Optimal ROI: ${roi2Mod.objVal:.2f}')


Print: $3000000.00
TV: $0.00
SEO: $0.00
AdWords: $1000000.00
Facebook: $3000000.00
LinkedIn: $3000000.00
Instagram: $0.00
Snapchat: $0.00
Twitter: $0.00
Email: $0.00
Optimal ROI: $456000.00


## Specific 5

#### Assuming the first ROI data is correct, if you were to use the second allocation (the allocation that assumed the second ROI data was correct) how much lower would the objective be relative to the optimal objective (the one that uses the first ROI data and the first allocation)?

In [7]:
total_roi = 0
for i in range(ncol):
    total_roi += (roi_data[roi1Mod.varName[i]][0] * roi2Mod.X[i])

print(f'It will be ${roi1Mod.objVal - total_roi:.2f} lower')

It will be $204000.00 lower


#### Assuming the second ROI data is correct, if you used the first allocation how much lower would the objective be relative to the optimal objective?

In [8]:
total_roi = 0
for i in range(ncol):
    total_roi += (roi_data[roi1Mod.varName[i]][1] * roi1Mod.X[i])

print(f'It will be ${roi2Mod.objVal - total_roi:.2f} lower')

It will be $192000.00 lower


#### Is the third Constraint Useful?

In [9]:
roi4Mod = gp.Model()

# Variables (removing constraint 3)
x = roi4Mod.addVars(platform, name = platform)

# # Objective function
roi4Mod.setObjective(gp.quicksum(roi_data[plat][0] * x[plat] for plat in platform), gp.GRB.MAXIMIZE)

# # Constraints
roi4Mod.addConstr(x.sum() <= 10e6) # Total budget constraint
roi4Mod.addConstr(x['Print'] + x['TV'] <= x['Facebook'] + x['Email']) # Constraint 1
roi4Mod.addConstr(x['Facebook'] + x['LinkedIn'] + x['Instagram'] + x['Snapchat'] + x['Twitter'] >= 2*(x['SEO'] + x['AdWords'])) # Constraint 2

# # Optimize
roi4Mod.Params.OutputFlag = 0
roi4Mod.optimize()

# # Result
for i in range(ncol):
    print(f'{roi4Mod.varName[i]}: ${roi4Mod.X[i]:.2f}')

print(f'Optimal ROI: ${roi4Mod.objVal:.2f}')

Print: $0.00
TV: $5000000.00
SEO: $0.00
AdWords: $0.00
Facebook: $0.00
LinkedIn: $0.00
Instagram: $0.00
Snapchat: $0.00
Twitter: $0.00
Email: $5000000.00
Optimal ROI: $465000.00


## Specific 6

In [10]:
df_sen = pd.DataFrame(columns = ["Platform", "ROI", "Allowable Decrease", "Allowable Increase"])

roi1Mod.optimize()

# Display sensitivity information
print("Platform, ROI, Allowable Decrease, Allowable Increase")
for v in roi1Mod.getVars():
    print(f'{v.varName}, {v.obj}, {v.SAObjLow - v.obj}, {v.SAObjUp - v.obj}')
    ser = pd.Series([v.varName,v.obj,v.SAObjLow - v.obj,v.SAObjUp - v.obj] , index = df_sen.columns)
    df_sen = pd.concat([df_sen,ser.to_frame().T], axis=0 , ignore_index = True)
    

Platform, ROI, Allowable Decrease, Allowable Increase
Print, 0.031, -inf, 0.018000000000000002
TV, 0.049, -0.010000000000000002, 0.012999999999999998
SEO, 0.024, -inf, 0.015
AdWords, 0.039, -0.005999999999999998, 0.006999999999999999
Facebook, 0.016, -inf, 0.012999999999999998
LinkedIn, 0.024, -inf, 0.015
Instagram, 0.046, -0.006999999999999999, inf
Snapchat, 0.026, -inf, 0.013000000000000001
Twitter, 0.033, -inf, 0.005999999999999998
Email, 0.044, -0.015, inf


In [11]:
df_sen

,Platform,ROI,Allowable Decrease,Allowable Increase
0,Print,0.031,-inf,0.018
1,TV,0.049,-0.01,0.013
2,SEO,0.024,-inf,0.015
3,AdWords,0.039,-0.006,0.007
4,Facebook,0.016,-inf,0.013
5,LinkedIn,0.024,-inf,0.015
6,Instagram,0.046,-0.007,inf
7,Snapchat,0.026,-inf,0.013
8,Twitter,0.033,-inf,0.006
9,Email,0.044,-0.015,inf


## Specifc 7

#### Your boss has gained permission to reinvest half of the return. For example, if the marketing obtains a 4% return in January, the budget of February will be $10M + $10M × 4% × 50% = $10.2M. The monthly ROI for next year is given in an attached csv file. The three constraints given by your boss are still in place for each month. What is the optimal allocation for each month?

In [12]:
# Starting budget for January
budget = 10000000  
platform_month = roi_data_month.columns.tolist()

print(platform_month)

for month in range(nrow_month):

    roi3Mod = gp.Model()

    # Variables (also setting the upper bound of each variable to $3M i.e. constraint 3)
    x = roi3Mod.addVars(platform_month, ub=3e6, name=platform_month)  # Allocation for each medium, constrained to $3M

    # Objective function
    roi3Mod.setObjective(gp.quicksum(roi_data_month[plat][month] * x[plat] for plat in platform_month)/100, gp.GRB.MAXIMIZE)

    # Constraints
    roi3Mod.addConstr(x['Print'] + x['TV'] <= x['Facebook'] + x['Email']) # Constraint 1
    roi3Mod.addConstr(x['Facebook'] + x['LinkedIn'] + x['Instagram'] + x['Snapchat'] + x['Twitter'] >= 2*(x['SEO'] + x['AdWords'])) # Constraint 2
    roi3Mod.addConstr(x.sum() <= budget) # Total budget constraint

    # Optimize
    roi3Mod.Params.OutputFlag = 0
    roi3Mod.optimize()
    
    # Results
    print(f"Optimal allocation for {months[month]}:")
    for i in range(ncol_month):
        print(f'{roi3Mod.varName[i]}: ${roi3Mod.X[i]:,.2f}')
    print(f"Optimal ROI for {months[month]}: {roi3Mod.objVal}")
    print()
    # Update budget for next month
    return_percentage = roi3Mod.objVal / budget
    budget = budget + budget * return_percentage * 0.5
    print('return pct = ' , return_percentage*100)
    print('budget for next month = ' , budget)
    print('===========================')


['Print', 'TV', 'SEO', 'AdWords', 'Facebook', 'LinkedIn', 'Instagram', 'Snapchat', 'Twitter', 'Email']
Optimal allocation for January:
Print: $3,000,000.00
TV: $0.00
SEO: $0.00
AdWords: $1,333,333.33
Facebook: $0.00
LinkedIn: $0.00
Instagram: $2,666,666.67
Snapchat: $0.00
Twitter: $0.00
Email: $3,000,000.00
Optimal ROI for January: 373000.0

return pct =  3.73
budget for next month =  10186500.0
Optimal allocation for February:
Print: $3,000,000.00
TV: $0.00
SEO: $0.00
AdWords: $2,395,500.00
Facebook: $3,000,000.00
LinkedIn: $0.00
Instagram: $0.00
Snapchat: $0.00
Twitter: $1,791,000.00
Email: $0.00
Optimal ROI for February: 406296.0

return pct =  3.9885731114710645
budget for next month =  10389648.0
Optimal allocation for March:
Print: $0.00
TV: $0.00
SEO: $0.00
AdWords: $3,000,000.00
Facebook: $0.00
LinkedIn: $3,000,000.00
Instagram: $1,389,648.00
Snapchat: $0.00
Twitter: $3,000,000.00
Email: $0.00
Optimal ROI for March: 414416.976

return pct =  3.9887489547287838
budget for next m

## Specific 8

Inside PDF